In [ ]:
%run appV2_dataCleaning.ipynb

In [ ]:
df

In [ ]:
#interesting tip:if chain format like tfidf= TfidfVectorizer(...).fit_transform() then 
#tfidf.get_feature_names doesnt work. However, if without chain like tfidf= TfidfVectorizer(...)
#tfidf.fit_transform(), then tfidf.get_feature_names works.

#conventional algor are often baised towards the majoirty class, not taking the data distribution into 
#consideration. One solution is to artificially balance the dataset for example oversampling each class. 

#classifier+learning algorithms can not directly process the text documents in their original form, 
#as most of them expect numerical feature vector with a fixed size rather than the raw text documents with
#variable length. One common approach for extracting features from text is to use the bag of words model:
#where for each document the presence(and often the freq) of words is taken into consideration. 
#Specifically, for each term in our dataset, we will calculate a measure called Term Freq Inverse Document 
#Freq (tf-idf) to calculate a tf-idf vector for each of comments.  
#tfidf = TfidfVectorizer(tokenizer= lemma, min_df=5, lowercase = True, ngram_range=(1,2), stop_words='english')
#vect_vectorized = tfidf.fit_transform(df['comment']) 


In [ ]:
#lemmatize the comment text instead of porter stemming
import pyforest
from nltk.stem import WordNetLemmatizer 
def lemma(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(w,'v') for w in tokens]
    return lemmatized

tfidf=TfidfVectorizer(tokenizer= lemma, min_df=3, lowercase = True, ngram_range=(1,2), stop_words='english')
features = tfidf.fit_transform(df.final).toarray()
labels = df.result
features.shape

interpretation_id_df = df[['interpretation', 'result']].sort_values('result')
interpretation_to_id = dict(interpretation_id_df.values)
id_to_interpretation = dict(interpretation_id_df[['result', 'interpretation']].values)

In [ ]:
interpretation_to_id

In [ ]:
len(np.unique(features.ravel()))

In [ ]:
#each comment is represented by 3956 features, representing the tf-idf score for different unigrams and bigrams. 
features.shape

In [ ]:
for interpretation, result in interpretation_to_id.items():
    print(interpretation, ' ', result)

In [ ]:
features[labels==1].shape

In [ ]:
from sklearn.feature_selection import chi2

features_chi2 = chi2(features, labels==0) 
score = features_chi2[0]
score = tuple(score)
score

In [ ]:
names = tuple(tfidf.get_feature_names())
names

In [ ]:
for i, (name, chi2) in enumerate(zip(tfidf.get_feature_names(), score)): 
    #if name == 'situ':
    print(i, '', name, '', chi2)

In [ ]:
#len([v for v in features_chi2[0] if np.isnan(v) == False]) 
idx = np.argsort(features_chi2[0])[::-1]

In [ ]:
#tfidf.get_feature_names()[0]
lst = np.array(tfidf.get_feature_names())[idx]  
lst

In [ ]:
[v for v in lst if len(v.split(' ')) == 1 ] #and v.split(' ')== 'situ'

In [ ]:
# use chi2 to find the unigram/bigram terms that are the most correlated with each of the category:
from sklearn.feature_selection import chi2

N = 20
for interpretation, result in interpretation_to_id.items():
    features_chi2 = chi2(features, labels==result) #return chi2 and p values
    #indices = np.asarray(features_chi2).max(0).argsort() 
    indices = np.argsort(features_chi2[0])#[::-1]
    feature_names = np.array(tfidf.get_feature_names())[indices]#tfidf has been fit and transformed.
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print(f'# {interpretation}') 
    ugrams = '\n. '.join(unigrams[-N:])
    print(f'  @@ Most correlated unigrams:\n. {ugrams}')   
    bgrams = '\n. '.join(bigrams[-N:])
    print(f'  @@ Most correlated bigrams:\n. {bgrams}')   

### name features: top 20 features that have the largest tf-idf
classifier+learning algorithms can not directly process the text documents in their original form, as most of them expect numerical feature vector with a fixed size rather than the raw text documents with variable length. One common approach for extracting features from text is to use the bag of words model: where for each document the presence(and often the freq) of words is taken into consideration. Specifically, for each term in our dataset, we will calculate a measure called Term Freq Inverse Document Freq (tf-idf) to calculate a tf-idf vector for each of comments.  

In [ ]:
tfidf = TfidfVectorizer(tokenizer= lemma, min_df=3, lowercase = True, ngram_range=(1,2), stop_words='english')
vect_vectorized = tfidf.fit_transform(df['final']) 

In [ ]:
sorted_tfidf_index = features.max(0).argsort()

In [ ]:
#a bag names
feature_names = np.array(tfidf.get_feature_names())

In [ ]:
feature_names

In [ ]:
#features now are all in a matrix. find max value along the rows. 
names = pd.Series(feature_names[vect_vectorized.max(0).toarray().ravel().argsort()], name = 'feature_names')

In [ ]:
sorted_tfidf_value = pd.Series(sorted(vect_vectorized.max(0).toarray().ravel().tolist()), name='tfidf', index = names)

In [ ]:
#vect_df = pd.concat([names, sorted_tfidf_value], axis=1)
#vect_df.set_index('feature_names') 

In [ ]:
sorted_tfidf_value.nlargest(20) # katherine needs to be removed. 

In [ ]:
names[names.str.contains('esophagu')]

In [ ]:
sorted_tfidf_value[sorted_tfidf_value.index.str.contains('esophagu')]

In [ ]:
#sorted_tfidf_value.index

In [ ]:
#historical code:
vect = TfidfVectorizer(lowercase = True, stop_words = 'english', ngram_range=(2,5) ).fit(df['comment'])
feature_names = np.array(vect.get_feature_names())
#transform the document into a do
vect_vectorized = vect.transform(df['comment']) 
sorted_tfidf_index = vect_vectorized.max(0).toarray().ravel().argsort()
names = pd.Series(feature_names[vect_vectorized.max(0).toarray().ravel().argsort()], name = 'feature_names')
sorted_tfidf_value = pd.Series(sorted(vect_vectorized.max(0).toarray().ravel().tolist()), name='tfidf')
vect_df = pd.concat([names, sorted_tfidf_value], axis=1)
small_df= vect_df.sort_values(by=['tfidf', 'feature_names']).iloc[:20,]
small_s=small_df['tfidf']
small_s.index=small_df['feature_names']
small_s.name=None
small_s.index.name=None
small_s
large_df = vect_df.sort_values(by=['tfidf', 'feature_names'], ascending = [False, True]).iloc[:20,] 
large_s = large_df['tfidf']
large_s.index=large_df['feature_names']
large_s.name = None
large_s.index.name = None

In [ ]:
sorted_tfidf_value = pd.Series(sorted(vect_vectorized.max(0).toarray().ravel().tolist()), name='tfidf')

In [ ]:
vect_df = pd.concat([names, sorted_tfidf_value], axis=1)

In [ ]:
large_df = vect_df.sort_values(by=['tfidf', 'feature_names'], ascending = [False, True]).iloc[:20,] 

In [ ]:
test_k = range(2, 100)[::5]
test_k 